## Setting up historical and current legislator data for parsing years of experience: send to csv (leg_histcurent.csv)

In [2]:
import yaml
import pandas as pd
from pandas.io.json import json_normalize

In [3]:
def read_yaml(fn):
    with open(fn, 'r') as fi:
        return yaml.load(fi)

def filter_data(data):
    result_data = []
    for x in data:
        if 'id' not in x:   continue
        if 'fec' not in x['id']:    continue
        if 'thomas' not in x['id']: continue
        result_data.append(x)
    return result_data


fn = 'legislators-historical.yaml'


df = json_normalize(filter_data(read_yaml(fn)), 'terms', [['id', 'fec',], ['id', 'thomas']])

df['id.fec'] = df['id.fec'].map(lambda x: str(x)[:-2])


df['id.fec'] = df['id.fec'].map(lambda x: str(x)[2:])



In [4]:
def read_yaml(fm):
    with open(fm, 'r') as fi:
        return yaml.load(fi)
def filter_data(data):
    result_data = []
    for x in data:
        if 'id' not in x:   continue
        if 'fec' not in x['id']:    continue
        if 'thomas' not in x['id']: continue
        result_data.append(x)
    return result_data
fm = 'legislators-current.yaml'
dy = json_normalize(filter_data(read_yaml(fm)), 'terms', [['id', 'fec',], ['id', 'thomas']])
dy['id.fec'] = dy['id.fec'].map(lambda x: str(x)[:-2])

dy['id.fec'] = dy['id.fec'].map(lambda x: str(x)[2:])

In [5]:
frames = [df,dy]
result = pd.concat(frames)


result.to_csv('setupforlegyears.csv', index=False)

tz = pd.read_csv('setupforlegyears.csv', usecols=['end', 'id.thomas', 'start'])

tz = tz.sort_values(['id.thomas', 'start',], ascending=(True, True))

tz.to_csv('leg_histcurent.csv')

## Merging years of experience to (billspon.legisl.merg): send to csv (billspon.legisl.merg1)

In [6]:
import pandas as pd
rr=pd.read_csv('Thesis/leg_years.csv', usecols=['years', 'id.thomas', \
                                               'served'])
rr['years_exp']=rr.groupby(['id.thomas']).served.cumsum()
rr.drop(['served'], axis=1, inplace=True)
rr = rr[(rr['years'] > 2004)]

zz=pd.read_csv('billspon.legisl.merg')

zz['years']=zz['date']
zz['years'] = zz['years'].astype('int')

zz = zz[(zz['years'] < 2015) & (zz['years'] > 2004)]

rf = pd.merge(zz, rr, on=['id.thomas', 'years'], how='left')



In [7]:
rf.to_csv('billspon_legisl_merg1', index=False)